In [1]:
!conda install pandas -y

Channels:
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\yuuds\miniconda3

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    bottleneck-1.4.2           |  py312h4b0e54e_0         145 KB
    conda-24.11.0              |  py312haa95532_0         1.2 MB
    intel-openmp-2023.1.0      |   h59b6b97_46320         2.7 MB
    mkl-2023.1.0               |   h6b88ed4_46358       155.9 MB
    mkl-service-2.4.0          |  py312h2bbff1b_1          55 KB
    mkl_fft-1.3.11             |  py312h827c3e9_0         169 KB
    mkl_random-1.2.8           |  py312h0158946_0         252 KB
    numexpr-2.10.1             |  py312h4cd664f_0         201 KB
    numpy-1.26.4               |  py312hfd52020_0          11 KB


WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(184): Could not remove or rename C:\Users\yuuds\miniconda3\Lib\site-packages\certifi\cacert.pem.  Please remove this file manually (you may need to reboot to free file handles)


In [8]:
import pandas as pd
import re

chord_list = [
    ['0', 'A', '', ''],
    ['1', '', 'A#', 'Bb'],
    ['2', 'B', '', ''],
    ['3', 'C', '', ''],
    ['4', '', 'C#', 'Db'],
    ['5', 'D', '', ''],
    ['6', '', 'D#', 'Eb'],
    ['7', 'E', '', ''],
    ['8', 'F', '', ''],
    ['9', '', 'F#', 'Gb'],
    ['10', 'G', '', ''],
    ['11', '', 'G#', 'Ab']
]
chord_df = pd.DataFrame(chord_list, columns=['key', 'natural', 'sharp', 'flat'])

# コード検出用の正規表現
chord_pattern = re.compile(r'\[([A-Ga-g#b]+[^/\]]*)(/[A-Ga-g#b]*)?\]')

# コードの移調関数
def transpose_chord(chord, semitones, sharp_flag, chord_df):
    root_match = re.match(r'([A-Ga-g#b]+)', chord)
    if not root_match:
        return chord
    root = root_match.group(1)

    # 現在のルートのkey値を取得
    root_row = chord_df[(chord_df['natural'] == root) | (chord_df['sharp'] == root) | (chord_df['flat'] == root)]
    if root_row.empty:
        return chord
    current_key = int(root_row.iloc[0]['key'])

    # 移調後のkey値を計算
    new_key = (current_key + semitones) % 12
    new_row = chord_df[chord_df['key'] == str(new_key)].iloc[0]

    # 移調後のルート名を選択
    if new_row['natural']:
        new_root = new_row['natural']
    else:
        new_root = new_row['sharp'] if sharp_flag == 1 else new_row['flat']

    # ルート以外の部分を保持
    suffix = chord[len(root):]
    return new_root + suffix

# 和音を移調する
def transpose_lyrics(lyrics_text, semitones, sharp_flag, chord_df):
    transposed_lyrics = lyrics_text

    def replace_chord(match):
        chord, bass = match.groups()
        transposed_chord = transpose_chord(chord, semitones, sharp_flag, chord_df)
        if bass:
            transposed_bass = transpose_chord(bass[1:], semitones, sharp_flag, chord_df)
            return f"[{transposed_chord}/{transposed_bass}]"
        return f"[{transposed_chord}]"

    transposed_lyrics = chord_pattern.sub(replace_chord, lyrics_text)
    return transposed_lyrics

In [6]:
lyrics = """
{key:F#}
[BM7]言い出せないことばかりが[A#m7]積もって
[BM7]真綿のような不安に[D#m7]悶えている[D#m7/E]
[BM7]捨てられないものばかりが[A#m7]残って
君の[BM7]合図を待ってい[D#m7]たんだ

[Cm7b5b9]た[CmM7]だ
{key:G}
[CM7]花火が声をかき[Bm7]消した
あ[Em7]りふれた日々が終[G7]わった
[CM7]さんざめく無数の[Bm7]光が
[Em7]轍をまた切り[G7]刻んだ
[CM7]俺はただ立ち尽[Bm7]くした
[Em7]俺はただ立ち尽[Em7/F]くした
[CM7]君のためにできる[Bm7]ことは何もなか[Em7]った[F#7]

{key:F#}
([BM7]----[A#m7]----[D#m7]----[F#7]----)×3
([BM7]----[A#m7]----[D#m7]----[D#7]----)
(書き刻んだ 枯れて夏は散った
日々は盲目に煌々と明滅 肌
発想は尽きない
想像が織り成す五百万画素の別天地を再現しよう
君が作った歌を聴いた
どんなものよりも美しい歌だと思ったんだ
ただ音楽が流れた
ただ音楽は流れた)

{key:Ab}
[DbM7]花火が[Cm7]散った
[Fm7]声が消[Ab7]えた
[DbM7]夏が終[Cm7]わった
[Fm7]青く染[Fm7/Gb]まった
[DbM7]花火が[Cm7]散った
[Fm7]声が消[Ab7]えた
[DbM7]夏が終[Cm7]わった
[Fm7]青く染[Fm7/Gb]まった

[DbM7]青く染[Cm7]まった
[Fm7]青く染[Ab7]まった
[DbM7]青く染[Cm7]まった
[Fm7]青く染[Ab7]まった
[DbM7]青く染[Cm7]まった
[Fm7]青く染[Ab7]まった
[DbM7]青く染[Cm7]まった
[Fm7]青く染[Ab7]まった

[DbM7]青く染まった
[DbM7]青く染まった
[DbM7]青く染まった
[DbM7]青く染まった
"""

transposed_lyrics = transpose_lyrics(lyrics, 1, 0, chord_df)
print(transposed_lyrics)


{key:F#}
[CM7]言い出せないことばかりが[m7]積もって
[CM7]真綿のような不安に[Em7]悶えている[Em7/F]
[CM7]捨てられないものばかりが[m7]残って
君の[CM7]合図を待ってい[Em7]たんだ

[Dbm7b5b9]た[DbmM7]だ
{key:G}
[DbM7]花火が声をかき[Cm7]消した
あ[Fm7]りふれた日々が終[Ab7]わった
[DbM7]さんざめく無数の[Cm7]光が
[Fm7]轍をまた切り[Ab7]刻んだ
[DbM7]俺はただ立ち尽[Cm7]くした
[Fm7]俺はただ立ち尽[Fm7/Gb]くした
[DbM7]君のためにできる[Cm7]ことは何もなか[Fm7]った[G7]

{key:F#}
([CM7]----[m7]----[Em7]----[G7]----)×3
([CM7]----[m7]----[Em7]----[E7]----)
(書き刻んだ 枯れて夏は散った
日々は盲目に煌々と明滅 肌
発想は尽きない
想像が織り成す五百万画素の別天地を再現しよう
君が作った歌を聴いた
どんなものよりも美しい歌だと思ったんだ
ただ音楽が流れた
ただ音楽は流れた)

{key:Ab}
[DM7]花火が[Dbm7]散った
[Gbm7]声が消[A7]えた
[DM7]夏が終[Dbm7]わった
[Gbm7]青く染[Gbm7/G]まった
[DM7]花火が[Dbm7]散った
[Gbm7]声が消[A7]えた
[DM7]夏が終[Dbm7]わった
[Gbm7]青く染[Gbm7/G]まった

[DM7]青く染[Dbm7]まった
[Gbm7]青く染[A7]まった
[DM7]青く染[Dbm7]まった
[Gbm7]青く染[A7]まった
[DM7]青く染[Dbm7]まった
[Gbm7]青く染[A7]まった
[DM7]青く染[Dbm7]まった
[Gbm7]青く染[A7]まった

[DM7]青く染まった
[DM7]青く染まった
[DM7]青く染まった
[DM7]青く染まった



In [10]:
lyrics = """
([Fm9]----[Bbm7sus4]----[Cm7]----[Gm7]----)*2

[Fm9]話をしよう[Bbm7sus4]
とても[Cm7]長い日々が[Gm7]暮れた

流星の[Fm9]かけらが
部品に[Bbm7sus4]なって時を[Cm7]回す[Gm7]

宝石が[Fm9]砂に
光が[Bbm7sus4]音に
変わる[Cm7]くらい長い[Gm7]日々

祈りが[Fm9]声に
願いが[Bbm7sus4]海に
変わる[Cm7]くらい長い[Gm7]日々

光が欲しい[Fm9]
光が欲しい[Bbm7sus4]
何も[Cm7]恐れ[Gm7]なくていい

光が欲しい[Fm9]
光が欲しい[Bbm7sus4]
何も[Cm7]遺せ[Gm7]なくていい

([Fm9]----[Bbm7sus4]----[Cm7]----[Gm7]----)*4

"""

transposed_lyrics = transpose_lyrics(lyrics, -1, 1, chord_df)
print(transposed_lyrics)


([Em9]----[Am7sus4]----[Bm7]----[F#m7]----)*2

[Em9]話をしよう[Am7sus4]
とても[Bm7]長い日々が[F#m7]暮れた

流星の[Em9]かけらが
部品に[Am7sus4]なって時を[Bm7]回す[F#m7]

宝石が[Em9]砂に
光が[Am7sus4]音に
変わる[Bm7]くらい長い[F#m7]日々

祈りが[Em9]声に
願いが[Am7sus4]海に
変わる[Bm7]くらい長い[F#m7]日々

光が欲しい[Em9]
光が欲しい[Am7sus4]
何も[Bm7]恐れ[F#m7]なくていい

光が欲しい[Em9]
光が欲しい[Am7sus4]
何も[Bm7]遺せ[F#m7]なくていい

([Em9]----[Am7sus4]----[Bm7]----[F#m7]----)*4




In [12]:
lyrics = """
[Db/Gb]振り切れたはずが気づけばこのザマだ[EM6]
[Db/Gb]忘れられんな　不細工なフライヤー[GM6]
[Db/Gb]言葉にしたらかなりくだらないよな[EM6]
[Db/Gb]変わらないまんま居られはしないかな[EM6][Bb9]

[Db/Gb]衛星
[Db/Ab]僕らは[Db/Bb]銘々街[Db/A]を行く
[Db/Gb]衛星
[Db/Ab]僕らの[Db/Bb]姿が[Cm7b5]見える[Db]か？

[Db/Gb]放せないまま
[Db/F]忘れないまま
[Db/Bb]立ち[Abm7]尽くして[Db7]る
[Db/Gb]狂えないまま
[Db/F]変われないまま
[Db/Bb]僕らは[Bb9]日々を過ごす

[Db/Gb]衛星
[Db/Ab]僕らは[Db/Bb]銘々街[Db/A]を行く
[Db/Gb]衛星
[Db/Ab]僕らの[Db/Bb]姿が[Cm7b5]見える[Db]か？

([Db/Gb]----[Db/F]----[Db/Bb]----[Abm7]--[Db7]--)*2
[Db/Gb]

"""

transposed_lyrics = transpose_lyrics(lyrics, 1, 1, chord_df)
print(transposed_lyrics)


[D/G]振り切れたはずが気づけばこのザマだ[FM6]
[D/G]忘れられんな　不細工なフライヤー[G#M6]
[D/G]言葉にしたらかなりくだらないよな[FM6]
[D/G]変わらないまんま居られはしないかな[FM6][B9]

[D/G]衛星
[D/A]僕らは[D/B]銘々街[D/A#]を行く
[D/G]衛星
[D/A]僕らの[D/B]姿が[C#m7b5]見える[D]か？

[D/G]放せないまま
[D/F#]忘れないまま
[D/B]立ち[Am7]尽くして[D7]る
[D/G]狂えないまま
[D/F#]変われないまま
[D/B]僕らは[B9]日々を過ごす

[D/G]衛星
[D/A]僕らは[D/B]銘々街[D/A#]を行く
[D/G]衛星
[D/A]僕らの[D/B]姿が[C#m7b5]見える[D]か？

([D/G]----[D/F#]----[D/B]----[Am7]--[D7]--)*2
[D/G]




In [14]:
lyrics = """
[Gb]

[Gb]散々だった日々が高速で[Gb/Cb]過ぎ去って[Gb/Db]いった
春風が[Gb/Eb]吹いた
[Gb]結局不安は全部空想で[Gb/Cb]杞憂に終[Gb/Db]わった
声が上[Gb/Eb]ずった

分[Gb/C]かれてし[Gb/B]まった月[Gb/Bb]がま[Gb/A]た重[Abm7]なる頃[Gb/G]僕らは[Gb]

話をし[CbM7]ようよ
笑えなくて[Gb/Db]いいよ
くだらなくて[Gb/Eb]いいよ
うまく話せ[Gb/Bb]ない[Gb/A]僕を見て[Abm7]ずっと[Gb/G]笑ってくれ[Gb]よ

[Gb]

[Gb]単純な話を複雑に[Gb/Cb]考えて悩[Gb/Db]んで何も分から[Gb/Eb]なくなって
[Gb]結局君はいつもそうやっ[Gb/Cb]て一人で考[Gb/Db]えて傷ついて[Gb/Eb]る

ぼ[Gb/C]やけて[Gb/B]まった月[Gb/Bb]がま[Gb/A]た重[Abm7]なる頃[Gb/G]僕らは[Gb]

許せなくて[CbM7]いいよ
変われなくて[Gb/Db]いいよ
話をし[Gb/Eb]ようよ
手の届か[Gb/Bb]ない背[Gb/A]中にずっ[Abm7]と焦[Gb/G]がれてい[Gb]た

([Cb/Gb]--------[Bb/Gb]--------)*2

話をし[Cb/Gb]ようよ
笑えなくていいよ
くだらなくて[Bb/Gb]いいよ

許せなくて[Cb/Gb]いいよ
変われなくていいよ
話をし[Bb/Gb]ようよ

分かれてし[CbM7]まった
月がまた[Gb/Db]いつか
1つに重[Gb/Eb]なるなら
迎えに行く[Gb/Bb]から
その時[CbM7]は
話をし[Gb/Db]ようよ
感情と感[Gb/Eb]覚の
話をし[Gb/Bb]ようよ

話をし[CbM7]ようよ

[CbM7]
"""

transposed_lyrics = transpose_lyrics(lyrics, 1, 0, chord_df)
print(transposed_lyrics)


[G]

[G]散々だった日々が高速で[G/Cb]過ぎ去って[G/D]いった
春風が[G/E]吹いた
[G]結局不安は全部空想で[G/Cb]杞憂に終[G/D]わった
声が上[G/E]ずった

分[G/Db]かれてし[G/C]まった月[G/B]がま[G/Bb]た重[Am7]なる頃[G/Ab]僕らは[G]

話をし[CbM7]ようよ
笑えなくて[G/D]いいよ
くだらなくて[G/E]いいよ
うまく話せ[G/B]ない[G/Bb]僕を見て[Am7]ずっと[G/Ab]笑ってくれ[G]よ

[G]

[G]単純な話を複雑に[G/Cb]考えて悩[G/D]んで何も分から[G/E]なくなって
[G]結局君はいつもそうやっ[G/Cb]て一人で考[G/D]えて傷ついて[G/E]る

ぼ[G/Db]やけて[G/C]まった月[G/B]がま[G/Bb]た重[Am7]なる頃[G/Ab]僕らは[G]

許せなくて[CbM7]いいよ
変われなくて[G/D]いいよ
話をし[G/E]ようよ
手の届か[G/B]ない背[G/Bb]中にずっ[Am7]と焦[G/Ab]がれてい[G]た

([Cb/G]--------[B/G]--------)*2

話をし[Cb/G]ようよ
笑えなくていいよ
くだらなくて[B/G]いいよ

許せなくて[Cb/G]いいよ
変われなくていいよ
話をし[B/G]ようよ

分かれてし[CbM7]まった
月がまた[G/D]いつか
1つに重[G/E]なるなら
迎えに行く[G/B]から
その時[CbM7]は
話をし[G/D]ようよ
感情と感[G/E]覚の
話をし[G/B]ようよ

話をし[CbM7]ようよ

[CbM7]



In [15]:
lyrics = """
[F]
ま[F/Bb]た新しい季節[F/C]が君を連れて
[F/D]変われない僕を[F/A]置いていく
[F/Bb]風にな[F/C]れたら[F/D]君のそ[F/A]ばに
[F/Bb]話したいこと[F/C]が増える度に
[F/D]もう会えないこと[F/A]に悲しくなる
[F/Bb]君に[F/C]何も[F/D]伝えきれ[F/A]なかった

[F/Bb]あいづちの[F/C]下手くそな[F/D]君と[F/A]話すのは[F/Bb]つ[F/C]まらないな[F/D]
[F/A]でも僕の[F/Bb]下手くそな[F/C]話を[F/D]聞いて[F/A]くれるの[F/Bb]は君[F/C]しかいなかった[F]

ま[F/Bb]た新しい季節[F/C]が君を連れて
[F/D]変われない僕を[F/A]置いていく
[F/Bb]風にな[F/C]れたら[F/D]君のそ[F/A]ばに
[F/Bb]話したいこと[F/C]が増える度に
[F/D]もう会えないこと[F/A]に悲しくなる
[F/Bb]君に[F/C]何も[F/D]伝えきれ[F/A]なかった

[F/Eb]----[F/D]----
許[F/Eb]せないのに[F/D]
[F/Eb]許されたいんだよ[F/D]
[F/Eb]君に何[F/D]も[F/Db]伝えきれなかった[F/C][F]

ま[F/Bb]た新しい季節[F/C]が君を連れて
[F/D]変われない僕を[F/A]置いていく
[F/Bb]風にな[F/C]っても[F/D]忘れな[F/A]いで
[F/Bb]話したいこと[F/C]が増える度に
[F/D]もう会えないこと[F/A]に悲しくなる
[F/Bb]君と[F/C]もっと[F/D]一緒に[F/A]いたかったんだ

[F/Bb]----[F/C]----[F/D]君に[F/A]何も
[F/Bb]伝えきれ[F/C]なかった[F/D]----[F]
"""

transposed_lyrics = transpose_lyrics(lyrics, 1, 1, chord_df)
print(transposed_lyrics)


[F#]
ま[F#/B]た新しい季節[F#/C#]が君を連れて
[F#/D#]変われない僕を[F#/A#]置いていく
[F#/B]風にな[F#/C#]れたら[F#/D#]君のそ[F#/A#]ばに
[F#/B]話したいこと[F#/C#]が増える度に
[F#/D#]もう会えないこと[F#/A#]に悲しくなる
[F#/B]君に[F#/C#]何も[F#/D#]伝えきれ[F#/A#]なかった

[F#/B]あいづちの[F#/C#]下手くそな[F#/D#]君と[F#/A#]話すのは[F#/B]つ[F#/C#]まらないな[F#/D#]
[F#/A#]でも僕の[F#/B]下手くそな[F#/C#]話を[F#/D#]聞いて[F#/A#]くれるの[F#/B]は君[F#/C#]しかいなかった[F#]

ま[F#/B]た新しい季節[F#/C#]が君を連れて
[F#/D#]変われない僕を[F#/A#]置いていく
[F#/B]風にな[F#/C#]れたら[F#/D#]君のそ[F#/A#]ばに
[F#/B]話したいこと[F#/C#]が増える度に
[F#/D#]もう会えないこと[F#/A#]に悲しくなる
[F#/B]君に[F#/C#]何も[F#/D#]伝えきれ[F#/A#]なかった

[F#/E]----[F#/D#]----
許[F#/E]せないのに[F#/D#]
[F#/E]許されたいんだよ[F#/D#]
[F#/E]君に何[F#/D#]も[F#/D]伝えきれなかった[F#/C#][F#]

ま[F#/B]た新しい季節[F#/C#]が君を連れて
[F#/D#]変われない僕を[F#/A#]置いていく
[F#/B]風にな[F#/C#]っても[F#/D#]忘れな[F#/A#]いで
[F#/B]話したいこと[F#/C#]が増える度に
[F#/D#]もう会えないこと[F#/A#]に悲しくなる
[F#/B]君と[F#/C#]もっと[F#/D#]一緒に[F#/A#]いたかったんだ

[F#/B]----[F#/C#]----[F#/D#]君に[F#/A#]何も
[F#/B]伝えきれ[F#/C#]なかった[F#/D#]----[F#]



In [18]:
lyrics = """
キー: A → Bb
チューニング: 半音下げ

い[DM7]つかは離れるこ[C#m7]とは分かっていた
行[DM7]かないでほしいと言[C#m7]えるわけもなかった

[DM7]繋いだ[C#m7]手が
[DM7]小さく[C#m7]震えていた
[DM7]何も言え[C#m7]ずに
[Bbm7]君を突き飛ばした
[Fm7b5]

{key:Bb}

The [EbM7]angel’s [Dm7]lost her wings[Gm7] [Fm7] [Bb7]
The [EbM7]angel’s [Dm7]lost her wings[Gm7] 
The [EbM7]angel’s [Dm7]lost her wings[Gm7] [Fm7] [Bb7]
So [EbM7]you should [Dm7]go anyway[Gm7]

[EbM7] [Dm7] [Gm7] [Fm7] [Bb7]
[EbM7] [Dm7] [Gm7] 
[EbM7] [Dm7] [Gm7] [Fm7] [Bb7]
[EbM7] [Dm7] [Gm7] 

The [EbM7]angel’s [Dm7]lost her wings[Gm7] [Fm7] [Bb7]
The [EbM7]angel’s [Dm7]lost her wings[Gm7] [Fm7] [Bb7]
The [EbM7]angel’s [Dm7]lost her wings[Gm7] [Fm7] [Bb7]
So [EbM7]you should [Dm7]go anyway[Gm7]

([EbM7] [Dm7] [Gm7] [Fm7] [Bb7]) *8
"""

transposed_lyrics = transpose_lyrics(lyrics, 1, 1, chord_df)
print(transposed_lyrics)


キー: A → Bb
チューニング: 半音下げ

い[D#M7]つかは離れるこ[Dm7]とは分かっていた
行[D#M7]かないでほしいと言[Dm7]えるわけもなかった

[D#M7]繋いだ[Dm7]手が
[D#M7]小さく[Dm7]震えていた
[D#M7]何も言え[Dm7]ずに
[Bm7]君を突き飛ばした
[F#m7b5]

{key:Bb}

The [EM7]angel’s [D#m7]lost her wings[G#m7] [F#m7] [B7]
The [EM7]angel’s [D#m7]lost her wings[G#m7] 
The [EM7]angel’s [D#m7]lost her wings[G#m7] [F#m7] [B7]
So [EM7]you should [D#m7]go anyway[G#m7]

[EM7] [D#m7] [G#m7] [F#m7] [B7]
[EM7] [D#m7] [G#m7] 
[EM7] [D#m7] [G#m7] [F#m7] [B7]
[EM7] [D#m7] [G#m7] 

The [EM7]angel’s [D#m7]lost her wings[G#m7] [F#m7] [B7]
The [EM7]angel’s [D#m7]lost her wings[G#m7] [F#m7] [B7]
The [EM7]angel’s [D#m7]lost her wings[G#m7] [F#m7] [B7]
So [EM7]you should [D#m7]go anyway[G#m7]

([EM7] [D#m7] [G#m7] [F#m7] [B7]) *8



In [19]:
lyrics = """
キー: Db
チューニング: 半音下げ

い[Db/Gb]つか[Db/Ab]は届[Db/Bb]くと思っていた
壊[Db/Gb]せ[Db/Ab]ないも[Db/Bb]のに気づいた
言[Db/Gb]葉[Db/Ab]が[Db/Bb]空を切って
そ[Db/Gb]の背中[Db/Ab]に届かない[Db/Bb]

([Db/Gb]---- [Db/Ab]---- [Db/Bb]----|----)*2

変[Db/Gb]わら[Db/Ab]ないも[Db/Bb]のもあった
細[Db/Gb]い髪[Db/Ab]が光[Db/Bb]を透かしていた
話[Db/Gb]せ[Db/Ab]ないこ[Db/Bb]とがあった
指[Db/Gb]先が[Db/Ab]触[Db/Bb]れた瞬間

[Db/Gb]純度無限大[Db/Ab]の異次元を描いた
[Db/Gb]純度無限大[Db/Ab]の異次元を[Db/Bb]探していた
[Db/Gb]純度無限大[Db/Ab]の異次元を[Db/Bb]掴もうとした
[Db/Gb]純度無限大[Db/Ab]の異次元に[Db/Bb]手を伸ばした

([Db/Gb]---- [Db/Ab]---- [Db/Bb]----|----)*5

[Db/Gb]放[Db/Ab]った光[Db/Bb]が
弾け出した想いが
[Db/Gb]瞬[Db/Ab]くよ[Db/Bb]
線を描いて

[Db/Gb]忘れ[Db/Ab]られない日々を
[Db/Bb]捨てていくよ
[Db/Gb]泣き腫[Db/Ab]らした君の
[Db/Bb]表情だって覚えてい[Db/Gb]るのに[Db/Ab]
[Db/Bb]表情だって覚えてい[Db/Gb]るのに[Db/Ab][Db/Bb]

([Db/Gb]---- [Db/Ab]---- [Db/Bb]----|----)*3
"""

transposed_lyrics = transpose_lyrics(lyrics, 1, 1, chord_df)
print(transposed_lyrics)


キー: Db
チューニング: 半音下げ

い[D/G]つか[D/A]は届[D/B]くと思っていた
壊[D/G]せ[D/A]ないも[D/B]のに気づいた
言[D/G]葉[D/A]が[D/B]空を切って
そ[D/G]の背中[D/A]に届かない[D/B]

([D/G]---- [D/A]---- [D/B]----|----)*2

変[D/G]わら[D/A]ないも[D/B]のもあった
細[D/G]い髪[D/A]が光[D/B]を透かしていた
話[D/G]せ[D/A]ないこ[D/B]とがあった
指[D/G]先が[D/A]触[D/B]れた瞬間

[D/G]純度無限大[D/A]の異次元を描いた
[D/G]純度無限大[D/A]の異次元を[D/B]探していた
[D/G]純度無限大[D/A]の異次元を[D/B]掴もうとした
[D/G]純度無限大[D/A]の異次元に[D/B]手を伸ばした

([D/G]---- [D/A]---- [D/B]----|----)*5

[D/G]放[D/A]った光[D/B]が
弾け出した想いが
[D/G]瞬[D/A]くよ[D/B]
線を描いて

[D/G]忘れ[D/A]られない日々を
[D/B]捨てていくよ
[D/G]泣き腫[D/A]らした君の
[D/B]表情だって覚えてい[D/G]るのに[D/A]
[D/B]表情だって覚えてい[D/G]るのに[D/A][D/B]

([D/G]---- [D/A]---- [D/B]----|----)*3

